2.ipynb：一份PDF文档，有三个（问题，答案）对 - 进阶RAG（检索器优化） - Ragas评估

- 仅embedding retriever -> bm25 retriever + embedding retriever = ensemble retriever

# 加载环境变量

In [1]:
import os
from dotenv import find_dotenv, load_dotenv

# 加载环境变量
load_dotenv(find_dotenv())

True

# 源数据

[2023全球智能汽车AI挑战赛——赛道一：AI大模型检索问答](https://tianchi.aliyun.com/competition/entrance/532154/forum)

# 构建RAG

In [2]:
def pretty_print_docs(docs):
  print(
      f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n + {d.page_content}\n{d.metadata}" for i,d in enumerate(docs)])
  )

In [3]:
from langchain_community.document_loaders import PyPDFLoader

# 加载文档
file_path = "data/初赛训练数据集.pdf"
loader = PyPDFLoader(file_path)
docs = loader.load()
print(len(docs))

354


In [4]:
pretty_print_docs(docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 
{'source': 'data/初赛训练数据集.pdf', 'page': 1}
----------------------------------------------------------------------------------------------------
Document 3:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
无线电设备 ..................

In [5]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 分割文档
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=50,
)
split_docs = text_splitter.split_documents(docs)
print(len(split_docs))

807


In [6]:
pretty_print_docs(split_docs[:3])

Document 1:

 + 欢迎
感谢您选择了具有优良安全性、舒适性、动力性和经济性的Lynk&Co领克汽车。
首次使用前请仔细、完整地阅读本手册内容，将有助于您更好地了解和使用车辆。
本手册中的所有资料均为出版时的最新资料，但本公司将对产品进行不断的改进和优化，您所购的车辆可能与本手册中的描述有所不同，请以实际
接收的车辆为准。
如您有任何问题，或需要预约服务，请拨打电话4006-010101 联系我们。您也可以开车前往Lynk &Co领克中心。
在抵达之前，请您注意驾车安全。©领克汽车销售有限公司
{'source': 'data/初赛训练数据集.pdf', 'page': 0}
----------------------------------------------------------------------------------------------------
Document 2:

 + 3目录
前言
本手册相关的重要信息 .................................................11
敬告用户.................................................................11
联系Lynk&Co领克 .....................................................12
{'source': 'data/初赛训练数据集.pdf', 'page': 2}
----------------------------------------------------------------------------------------------------
Document 3:

 + 事件数据记录系统 ......................................................12
远程监控系统............................................................12
原厂精装附件、选装装备和改装 ......................................13
{'source': 'data/初赛训练数据集.

In [7]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

# 创建嵌入模型
model_name = 'BAAI/bge-large-zh-v1.5'
model_kwargs = {'device': 'cuda'}  # 需要安装GPU版本的torch 
encode_kwargs = {'normalize_embeddings': True}
embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
)

D:\Software\anaconda3\envs\py38_AdvancedRAG\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
from langchain_community.vectorstores import FAISS

# 创建向量存储
vectordb = FAISS.from_documents(split_docs, embeddings)

In [9]:
index_folder_path = "data/faiss_index"
index_name = "2"

In [10]:
# 保存索引
vectordb.save_local(index_folder_path, index_name)

In [11]:
# 加载索引
vectordb = FAISS.load_local(index_folder_path, embeddings, index_name)

In [12]:
# 创建密集检索器，擅长根据语义相似度查找相关文档
faiss_retriever = vectordb.as_retriever(search_kwargs={"k": 10})

In [13]:
from langchain.retrievers import BM25Retriever, EnsembleRetriever

# 创建稀疏检索器，擅长根据关键词查找相关文档
bm25_retriever = BM25Retriever.from_documents(split_docs)
bm25_retriever.k=10

# 创建混合检索器
retriever = EnsembleRetriever(retrievers=[bm25_retriever, faiss_retriever], 
                              weight=[0.5, 0.5])

测试

In [14]:
question = "怎么打开危险警告灯？"
retrieval_docs = retriever.invoke(question)
len(retrieval_docs)

20

In [15]:
# 去重后
len(list(set([doc.page_content for doc in retrieval_docs])))

20

In [16]:
pretty_print_docs(retrieval_docs)

Document 1:

 + 技术资料
354公制术语
术语 说明
% 百分比
X:1 比值
℃ 摄氏温度
Ah 安时
m 米
cm 厘米
mm 毫米
g 克
kg 千克
h 小时
min 分钟
s 秒
rpm 每分钟转数
km/h 千米每小时
L 升
mL 毫升术语 说明
N 牛
Nm 牛米
V 伏特
W 瓦特
kPa 千帕
kW 千瓦
{'source': 'data/初赛训练数据集.pdf', 'page': 353}
----------------------------------------------------------------------------------------------------
Document 2:

 + 使用危险警告灯 ........................................................ 89
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 3:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指示灯：打开近光灯时，该指示灯点亮。
远光灯指示灯：打开远光灯时，该指示灯点亮。
智能远近光控制开启指示灯：打开智能远近光控制时，该
指示灯点亮。
后雾灯指示灯：打开后雾灯时，该指示灯点亮。
位置灯指示灯：打开位置灯时，该指示灯点亮。
泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点
亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 73}
----------------------------------------------------------------------------------------------------
Document 4:

 + 安全出行
110打开/关闭全景天窗
全景天窗由两部分组成，您可以使用车顶控制面板上的控制按钮，起
翘/滑动前半部分。后半部分为固定部分。
全景天窗配备有遮阳帘，位于车顶玻璃下方，在阳光强烈的情况下提


In [17]:
pretty_print_docs(faiss_retriever.invoke(question))

Document 1:

 + 使用危险警告灯 ........................................................ 89
{'source': 'data/初赛训练数据集.pdf', 'page': 3}
----------------------------------------------------------------------------------------------------
Document 2:

 + 指示灯闪烁。打开危险警告灯开关时，两侧转
向指示灯均闪烁。
近光灯指示灯：打开近光灯时，该指示灯点亮。
远光灯指示灯：打开远光灯时，该指示灯点亮。
智能远近光控制开启指示灯：打开智能远近光控制时，该
指示灯点亮。
后雾灯指示灯：打开后雾灯时，该指示灯点亮。
位置灯指示灯：打开位置灯时，该指示灯点亮。
泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点
亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 73}
----------------------------------------------------------------------------------------------------
Document 3:

 + 灯点亮。
陡坡缓降系统故障警告灯：陡坡缓降系统出现故障时，该
警告灯点亮。
安全气囊故障警告灯：安全气囊系统或预紧器系统存在故
障时，该警告灯点亮。
{'source': 'data/初赛训练数据集.pdf', 'page': 72}
----------------------------------------------------------------------------------------------------
Document 4:

 + 仪表和灯光
89
01点击开启/关闭门控灯。
使用超车灯
超车时，您可将拨杆朝向自身方向拉动然后松开，此时远光灯闪烁一
次，以提醒前方车辆。
使用危险警告灯
危险警告灯按键
车辆遇到交通事故或其他紧急情况时，按下危险警告灯按键，启用危
险警告灯。
说明！
□发生碰撞情况下，危险警告灯也将自动点亮。
{'source': 'data/初赛训练数据集.pdf', '

In [18]:
pretty_print_docs(bm25_retriever.invoke(question))

Document 1:

 + 技术资料
354公制术语
术语 说明
% 百分比
X:1 比值
℃ 摄氏温度
Ah 安时
m 米
cm 厘米
mm 毫米
g 克
kg 千克
h 小时
min 分钟
s 秒
rpm 每分钟转数
km/h 千米每小时
L 升
mL 毫升术语 说明
N 牛
Nm 牛米
V 伏特
W 瓦特
kPa 千帕
kW 千瓦
{'source': 'data/初赛训练数据集.pdf', 'page': 353}
----------------------------------------------------------------------------------------------------
Document 2:

 + 安全出行
110打开/关闭全景天窗
全景天窗由两部分组成，您可以使用车顶控制面板上的控制按钮，起
翘/滑动前半部分。后半部分为固定部分。
全景天窗配备有遮阳帘，位于车顶玻璃下方，在阳光强烈的情况下提
供保护。
将点火开关转至 I或以上挡位，可以操作全景天窗和遮阳帘。
打开/关闭全景天窗
手动滑动打开（轻按按钮至第1个停止位置）。
自动滑动打开（按到底）。
手动滑动关闭（轻按按钮至第1个停止位置）。
自动滑动关闭（按到底）。
如果全景天窗和遮阳帘处于完全关闭状态，轻按控制按钮，先打开遮
{'source': 'data/初赛训练数据集.pdf', 'page': 109}
----------------------------------------------------------------------------------------------------
Document 3:

 + 安全出行
114
2点击
、
等指示箭头，调节选择的座椅功能。
前排座椅加热
通过中央显示屏调节
设计前排座椅加热功能的目的在于通过加热前排座椅，提高环境舒适
度。
您可以通过中央显示屏，设置驾驶员/副驾驶员侧座椅加热强度或关
闭座椅加热功能。
在中央显示屏中点击
 ，进入驾驶员侧座椅加热控制界面。
01设置驾驶员侧座椅加热强度及开关控制。
在中央显示屏中点击
 ，进入副驾驶员侧座椅加热控制界面。
{'source': 'data/初赛训练数据集.pdf', 'page': 113}
----------------

In [19]:
from langchain_openai import ChatOpenAI

# 创建模型
llm = ChatOpenAI(temperature=0)
print(llm.model_name)

gpt-3.5-turbo


In [20]:
from langchain.chains import RetrievalQA

# 创建链
chain = RetrievalQA.from_chain_type(llm=llm,
                                 chain_type="stuff",
                                 retriever=retriever,
                                 return_source_documents=True)

In [26]:
# # 运行链 - 测试
# question = "怎么打开危险警告灯？"
# response = chain(question)
# print(response)

In [27]:
# response['query']

In [28]:
# response['result']

In [29]:
# pretty_print_docs(response['source_documents'])

# 评估RAG

[使用您的测试集进行评估](https://docs.ragas.io/en/stable/getstarted/evaluation.html)

数据集包含以下列：

- question: list[str] - 这些是将评估您的 RAG 管道的问题。
- answer: list[str] - 您的 RAG 管道生成的答案。
- context: list[list[str]] - 传递到 LLM 来回答问题的上下文。
- ground_truth: list[str] - 问题的真实答案。


In [21]:
questions = ["怎么打开危险警告灯？",
            "车辆如何保养？",
            "靠背太热怎么办？"]

ground_truths = ["危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。",
            "为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮胎的保养、低压蓄电池的保养等。",
            "您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→座椅→加热，在该界面下可以关闭座椅加热。"]

In [22]:
# 生成 answers 和 contexts
answers = []
contexts = []

for question in questions:
    print(question)
    response = chain(question)
    print(response['result'], "\n")
    answers.append(response['result'])
    contexts.append([doc.page_content for doc in response['source_documents']])

怎么打开危险警告灯？
要打开危险警告灯，您需要按下车辆内的危险警告灯按键。通常，这个按键位于车辆的中控面板上，可能会有一个三角形符号或者写有“HAZARD”的标识。当您按下这个按键时，危险警告灯会开始闪烁，以提醒其他车辆和行人注意到您的车辆处于紧急情况下。 

车辆如何保养？
根据提供的信息，以下是一些关于车辆保养的建议：
1. 定期清洗车辆外部，避免使用含酸清洁剂、强碱性肥皂、强化学性清洗剂、汽油或溶剂清洗车辆，以防损坏车辆表面。
2. 在冬季道路撒盐的地区，应定期清洁车底，以防止盐累积导致腐蚀。
3. 清洗完车辆后，将车辆表面擦拭干净，避免残留的清洁剂腐蚀车辆外观。
4. 定期存放车辆在阴凉通风、清洁干燥的环境下，避免长期停放在封闭潮湿环境中导致部件生锈老化。
5. 定期更换遥控钥匙电池，以确保遥控钥匙正常工作。
6. 定期检查轮胎是否损坏，保持正确的充气压力，避免让轮胎接触到润滑脂、润滑油和燃油。
7. 定期保养发动机、动力传动系统等车辆系统，按照规定的保养周期更换机油、空气滤芯、火花塞等零部件。
8. 定期清洁车辆内饰，使用推荐的清洁剂和保养产品进行清理。

这些是一些基本的车辆保养建议，具体的保养项目和周期可能会根据车辆型号和使用情况而有所不同。如果需要更详细的信息，建议查阅车辆的用户手册或咨询专业的维修技师。 

靠背太热怎么办？
如果座椅靠背感觉太热，您可以尝试以下方法来解决问题：
1. 调节座椅加热功能：通过中央显示屏调节座椅加热功能的强度或关闭座椅加热功能，以降低座椅的温度。
2. 调整座椅的位置：尝试调整座椅的位置，可能会改变座椅靠背与加热元件的接触面积，从而减少热度感觉。
3. 确保座椅通风良好：确保座椅周围通风良好，避免座椅靠背过热。
如果以上方法无法解决问题，建议联系车辆制造商或经销商进行进一步的检查和维修。 



In [23]:
# 构建评估数据集
from datasets import Dataset

evaluate_data = {
    "question": questions,
    "answer": answers,
    "contexts": contexts,
    "ground_truth": ground_truths
}

evaluate_dataset = Dataset.from_dict(evaluate_data)
evaluate_dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truth'],
    num_rows: 3
})

In [24]:
evaluate_dataset[0]

{'question': '怎么打开危险警告灯？',
 'answer': '要打开危险警告灯，您需要按下车辆内的危险警告灯按键。通常，这个按键位于车辆的中控面板上，可能会有一个三角形符号或者写有“HAZARD”的标识。当您按下这个按键时，危险警告灯会开始闪烁，以提醒其他车辆和行人注意到您的车辆处于紧急情况下。',
 'contexts': ['技术资料\n354公制术语\n术语 说明\n% 百分比\nX:1 比值\n℃ 摄氏温度\nAh 安时\nm 米\ncm 厘米\nmm 毫米\ng 克\nkg 千克\nh 小时\nmin 分钟\ns 秒\nrpm 每分钟转数\nkm/h 千米每小时\nL 升\nmL 毫升术语 说明\nN 牛\nNm 牛米\nV 伏特\nW 瓦特\nkPa 千帕\nkW 千瓦',
  '使用危险警告灯 ........................................................ 89',
  '指示灯闪烁。打开危险警告灯开关时，两侧转\n向指示灯均闪烁。\n近光灯指示灯：打开近光灯时，该指示灯点亮。\n远光灯指示灯：打开远光灯时，该指示灯点亮。\n智能远近光控制开启指示灯：打开智能远近光控制时，该\n指示灯点亮。\n后雾灯指示灯：打开后雾灯时，该指示灯点亮。\n位置灯指示灯：打开位置灯时，该指示灯点亮。\n泊车紧急制动指示灯：泊车紧急制动激活时，该指示灯点\n亮。',
  '安全出行\n110打开/关闭全景天窗\n全景天窗由两部分组成，您可以使用车顶控制面板上的控制按钮，起\n翘/滑动前半部分。后半部分为固定部分。\n全景天窗配备有遮阳帘，位于车顶玻璃下方，在阳光强烈的情况下提\n供保护。\n将点火开关转至 I或以上挡位，可以操作全景天窗和遮阳帘。\n打开/关闭全景天窗\n手动滑动打开（轻按按钮至第1个停止位置）。\n自动滑动打开（按到底）。\n手动滑动关闭（轻按按钮至第1个停止位置）。\n自动滑动关闭（按到底）。\n如果全景天窗和遮阳帘处于完全关闭状态，轻按控制按钮，先打开遮',
  '安全出行\n114\n2点击\n、\n等指示箭头，调节选择的座椅功能。\n前排座椅加热\n通过中央显示屏调节\n设计前排座椅加热功能的目的在于通过加热前排座椅，提高环境舒适\n度。\n您可以通过中央显示屏，设置驾驶员/副驾驶


[评估指标](https://docs.ragas.io/en/stable/concepts/metrics/index.html)
- Faithfulness
  - 如果答案中提出的所有主张都可以从给定的上下文中推断出来，则生成的答案被认为是忠实的。
  - answer 和 contexts
- Answer Relevancy
  - 评估指标“答案相关性”重点评估生成的答案与给定提示的相关程度。不完整或包含冗余信息的答案将获得较低分数。
  - answer 和 question
- Context Precision
  - 用于评估 contexts 中存在的所有真实相关项目是否排名较高。理想情况下，所有相关块必须出现在顶层。
  - question 和 contexts
- Context Recall
  - 上下文回忆衡量检索到的上下文与带注释的答案（被视为基本事实）的一致程度。
  - ground truth 和 contexts
---

- RAGAS评估框架评估了RAG管道的两个主要组件：
    - Retriever 检索器
    - Generator 生成器

---

- 与评估 Retrieval 相关的指标如下：
    - Context Precision。question 和 contexts
    - Context Recall。ground truth 和 contexts
- 与评估 Generation 相关的指标如下：
  - Faithfulness。answer 和 contexts
  - Answer Relevancy。answer 和 question

In [25]:
# 导入指标
from ragas.metrics import (
    faithfulness,
    answer_relevancy,
    context_recall,
    context_precision,
)


# 开始评估
from ragas import evaluate

evaluate_result = evaluate(
    evaluate_dataset, 
    metrics=[
        faithfulness,
        answer_relevancy,
        context_recall,
        context_precision,
    ]
)
evaluate_result

Evaluating: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 12/12 [00:42<00:00,  3.58s/it]


{'faithfulness': 1.0000, 'answer_relevancy': 0.8593, 'context_recall': 1.0000, 'context_precision': 0.1517}

In [26]:
df_evaluate_result = evaluate_result.to_pandas()
df_evaluate_result

,question,answer,contexts,ground_truth,faithfulness,answer_relevancy,context_recall,context_precision
0,怎么打开危险警告灯？,要打开危险警告灯，您需要按下车辆内的危险警告灯按键。通常，这个按键位于车辆的中控面板上，可能...,[技术资料\n354公制术语\n术语 说明\n% 百分比\nX:1 比值\n℃ 摄氏温度\n...,危险警告灯开关在方向盘下方，按下开关即可打开危险警告灯。,1.0,0.880637,1.0,0.125
1,车辆如何保养？,根据提供的信息，以下是一些关于车辆保养的建议：\n1. 定期清洗车辆外部，避免使用含酸清洁剂...,[技术资料\n354公制术语\n术语 说明\n% 百分比\nX:1 比值\n℃ 摄氏温度\n...,为了保持车辆处于最佳状态，建议您定期关注车辆状态，包括定期保养、洗车、内部清洁、外部清洁、轮...,1.0,0.844589,1.0,0.330
2,靠背太热怎么办？,如果座椅靠背感觉太热，您可以尝试以下方法来解决问题：\n1. 调节座椅加热功能：通过中央显示...,[安全出行\n114\n2点击\n、\n等指示箭头，调节选择的座椅功能。\n前排座椅加热\n...,您好，如果您的座椅靠背太热，可以尝试关闭座椅加热功能。在多媒体显示屏上依次点击空调开启按键→...,1.0,0.852783,1.0,0.000


In [27]:
df_evaluate_result.to_excel(f'data/{index_name}_eval_result.xlsx', index=False)